# Notebook for analyzing the results obtained from larger networks

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from bnsl.metrics import compute_shd
from typing import List, Tuple
from math import log10, floor

In [ ]:
root = Path.cwd().parents[1]    # two levels up
files = list((root / "data" / "results" / "medium").rglob("*.json"))

In [ ]:
records = []
for f in files:
    with open(f) as fp:
        r = json.load(fp)
        record = {
            "algorithm": r["algorithm"],
            "network": r["network"].split("/")[-1].split(".")[0],
            "num_samples": r["num_samples"],
            "score": r["score"],
            "theoretical_upper_bound": r["bounds"].get("theoretical_upper_bound"),
            "naive_upper_bound": r["bounds"].get("naive_upper_bound"),
            "runtime": r["seconds_elapsed"],
            "k": r["params"].get("k"),
            "l": r["params"].get("l"),
            "num_vars": r["num_variables"],
            "seed": r["seed"],
            "parent_map": r.get("parent_map"),
        }
        records.append(record)

df = pd.DataFrame(records)

In [ ]:
df.head()

##### Look at the number of ideals generated for each partial order 

In [ ]:
def get_size_of_ideals(n:int, l: int, k: int) -> int:
    """ Returns the expected number of ideals for a partial order, given l and k values.
     Based on lemma 19 from Partial Order Approach paper: |I(B)| = 1−ℓ+2**|B1|+2**|B2|+···+2**|Bℓ|"""
    
    q, r = divmod(n, k) #partition n into k equally sized sets:  r sets of size q+1, (k-r) sets  of size q

    if l <= r:
        size_big = l * (q + 1)

        num_small_q_plus_1 = r - l 
        num_small_q = k - r  
        L = num_small_q_plus_1 + num_small_q + 1

        return (1- L+ num_small_q_plus_1 * (2 ** (q + 1)) + num_small_q * (2 ** q) + (2 ** size_big))
    else: # l > r
        size_big = r * (q + 1) + (l - r) * q

        num_small_q = k - l 
        L = num_small_q + 1

        return (1- L+ num_small_q * (2 ** q) + (2 ** size_big))

In [ ]:
networks_sorted = [
    ("CHILD", 20),
    ("INSURANCE", 27),
    ("WATER", 32),
    ("MILDEW", 35),
    ("ALARM", 37),
    ("HAILFINDER", 56),
    ("CARPO", 60),
]

k_l_grid: List[Tuple[int, int]] = [
    (2, 1),
    (3, 2),
    (4, 3),
    (5, 4),
]

def to_sci_latex(x: int) -> str:
    """
    Format integer x  in latex 
    """
    if x == 0:
        return "$0$"
    exp = floor(log10(x))
    mant = x / (10 ** exp)
    # 2 decimal digits 
    return f"${mant:.2f} \\times 10^{{{exp}}}$"

# Print LaTeX table
print(r"\begin{table}[H]")
print(r"\centering")
print(r"\begin{tabular}{l r " + "r" * len(k_l_grid) + "}")
print(r"\toprule")

# Header row
header = [r"\textbf{Network}", r"\textbf{Nodes $n$}"]
for k, l in k_l_grid:
    header.append(fr"$\mathbf{{(k={k},\,\ell={l})}}$")
print(" & ".join(header) + r" \\")
print(r"\midrule")

# Data rows
for name, n in networks_sorted:
    row = [name, str(n)]
    for k, l in k_l_grid:
        ideals_size = get_size_of_ideals(n, l, k)
        row.append(to_sci_latex(ideals_size))
    print(" & ".join(row) + r" \\")
print(r"\bottomrule")
print(r"\end{tabular}")
print(r"\caption{Number of ideals generated by the first partial order "
      r"for different $(k,\ell)$ values.}")
print(r"\end{table}")

##### Plot analytical score against empirical runtime

In [ ]:
df_avg = (
    df.groupby(["network", "num_samples", "k", "l", "num_vars"])
      .agg(runtime=("runtime", "mean"))
      .reset_index()
)

df_avg["theory_log"] = (df_avg["l"] / df_avg["k"]) * df_avg["num_vars"] # log_2(2**(l/k * n)) = (l/k)*n
df_avg["actual_log"] = np.log2(df_avg["runtime"])

networks = df_avg["network"].unique()

for net in networks:
    subset = df_avg[df_avg["network"] == net]

    plt.figure(figsize=(6, 4))

    # one scatter series per (k, l)
    for (k, l), sub2 in subset.groupby(["k", "l"]):
        plt.scatter(
            sub2["theory_log"],
            sub2["actual_log"],
            label=f"k={k}, ℓ={l}",
            alpha=0.8,
        )

    plt.title(f"Runtime scaling for {net}-network")
    plt.xlabel(r"$\log(\text{theoretical runtime})$")
    plt.ylabel(r"$\log(\text{empirical runtime})$")
    plt.legend(title="(k, ℓ)")
    plt.tight_layout()

    outfile = root / f"experiments/plots/runtime_scaling_{net}.png"
    plt.savefig(outfile, dpi=300) 
    print(f"Saved {outfile}")
    plt.show()

##### Score vs runtime

In [ ]:
df_agg = (
    df.groupby(["num_samples", "network", "k", "l"], as_index=False)
      .agg(
          score=("score", "mean"),
          naive_upper_bound=("naive_upper_bound", "mean"),
      )
)

In [ ]:
# Filter to a single sample size
TARGET_N = 100
df_1000 = df_agg[df_agg["num_samples"] == TARGET_N].copy()

# Add approximation ratio
df_1000["ratio"] = df_1000["l"] / df_1000["k"]

# Plot one figure per network
for net, sub in df_1000.groupby("network"):
    plt.figure(figsize=(6, 4))

    # Sort by ratio so the line is monotone
    sub = sub.sort_values("ratio")

    x = sub["ratio"].to_numpy()
    y = sub["score"].to_numpy()

    # Approximation score curve
    plt.plot(
        x,
        y,
        marker="o",
        linestyle="-",
        label=f"Approx score (n={TARGET_N})"
    )



    kl_list = [tuple(x) for x in sub[["k", "l"]].drop_duplicates().to_numpy()]
    kl_list = sorted(kl_list)

    xticks = [l / k for (k, l) in kl_list]
    xticklabels = [fr"$\frac{{{int(l)}}}{{{int(k)}}}$" for (k, l) in kl_list]

    plt.xticks(xticks, xticklabels)

    plt.xlabel(r"Approximation ratio $\frac{l}{k}$")
    plt.ylabel("Score")
    plt.title(f"Score vs. approximation ratio ({net}, n={TARGET_N})")
    plt.legend(fontsize=7)
    plt.tight_layout()

    outfile = root / f"experiments/plots/plot_score_vs_ratio_{net}_n{TARGET_N}.png"
    plt.show()

##### Time vs ratio 

In [ ]:
df_avg = (
    df.groupby(["network", "num_samples", "k", "l", "num_vars"])
      .agg(runtime=("runtime", "mean"))
      .reset_index()
)

df_avg["ratio"] = df_avg["l"] / df_avg["k"]

# Compute base c such that runtime ≈ c^n
df_avg["exp_base"] = df_avg["runtime"] ** (1 / df_avg["num_vars"])

networks = df_avg["network"].unique()

for net in networks:
    subset_net = df_avg[df_avg["network"] == net]
    sample_sizes = subset_net["num_samples"].unique()

    plt.figure(figsize=(7, 5))

    for S in sorted(sample_sizes):
        sub = subset_net[subset_net["num_samples"] == S].sort_values("ratio")

        plt.plot(
            sub["ratio"],
            sub["exp_base"],
            marker="o",
            label=f"N={S}"
        )

    plt.xlabel(r"Approximation ratio $\ell/k$")
    plt.ylabel(r"Base of exponential runtime")

    plt.title(f"Empirical exponential base vs approximation ratio ({net})")
    plt.legend(title="Sample size")
    plt.tight_layout()

    outfile = root / f"experiments/plots/exp_base_vs_ratio_{net}.png"
    plt.savefig(outfile, dpi=300)
    print(f"Saved {outfile}")

    plt.show()


##### Score vs runtime

In [ ]:
S = 1000

df_fixed = df[df["num_samples"] == S]

df_avg = (
    df_fixed.groupby(["network", "k", "l"])
            .agg(runtime=("runtime", "mean"),
                 score=("score", "mean"))
            .reset_index()
)

df_avg["log_runtime"] = np.log(df_avg["runtime"])

networks = df_avg["network"].unique()

for net in networks:
    subset = df_avg[df_avg["network"] == net]

    plt.figure(figsize=(6, 4))

    for (k, l), sub2 in subset.groupby(["k", "l"]):
        plt.scatter(
            sub2["log_runtime"],
            sub2["score"],
            label=f"k={k}, ℓ={l}",
            alpha=0.8,
        )

    plt.title(f"Score vs. runtime for {net}-network (N={S})")
    plt.xlabel(r"$\log(\text{runtime})$")
    plt.ylabel("Score (BIC)")
    plt.legend(title="(k, ℓ)")
    plt.tight_layout()

    outfile = root / f"experiments/plots/score_vs_runtime_{net}_N{S}.png"
    plt.savefig(outfile, dpi=300)
    print(f"Saved {outfile}")

    plt.show()


##### Hailfinder!

In [ ]:
files = list((root / "data" / "results" / "large").rglob("*.json"))
records = []
for f in files:
    with open(f) as fp:
        r = json.load(fp)
        record = {
            "algorithm": r["algorithm"],
            "network": r["network"].split("/")[-1].split(".")[0],
            "num_samples": r["num_samples"],
            "score": r["score"],
            "theoretical_upper_bound": r["bounds"].get("theoretical_upper_bound"),
            "naive_upper_bound": r["bounds"].get("naive_upper_bound"),
            "runtime": r["seconds_elapsed"],
            "k": r["params"].get("k"),
            "l": r["params"].get("l"),
            "num_vars": r["num_variables"],
            "seed": r["seed"],
            "parent_map": r.get("parent_map"),
        }
        records.append(record)

df_large = pd.DataFrame(records)

In [ ]:
df_large.head(n=10)

In [ ]:
from pgmpy.readwrite import BIFReader
def index_pm_to_name_pm(pm_idx, network_path):
    reader = BIFReader(str(network_path))
    model = reader.get_model()
    var_names = list(model.nodes())
    index_to_name = {i: name for i, name in enumerate(var_names)}
    
    return {
        index_to_name[int(child)]: {index_to_name[int(p)] for p in parents}
        for child, parents in pm_idx.items()
    }


In [ ]:
network_path = root / "networks" / "large" / "hailfinder.bif"

pm_idx = df_large[
    (df_large["network"] == "hailfinder") &
    (df_large["num_samples"] == 10000)
]["parent_map"].iloc[0]

pm_named = index_pm_to_name_pm(pm_idx, network_path)

compute_shd(network_path, pm_named)
